### Calculating Null Distribution



Null distribution - is generated by getting the median correlation score of randomly combined replicates that do not come from the same compounds.



### The goal here: 

-- is to compute the **p-value** for each compound per dose by evaluating the probability of random combinations of replicates (from different compounds) having greater median correlation score than replicates that come from the same compound.




- In our case, we generated 1000 median correlation scores from randomly combined replicates as the **null distribution** for each no_of_replicates/replicate class per DOSE i.e. for a no_of_replicates class for every DOSE (1-6) - we have 1000 medians scores from randomly combined replicates of different compounds.





**no_of_replicate** is the number of replicates in a specific compound and **no_of_replicate class** is a specific group of compounds that have the same amount of replicates e.g all compounds with 3 replicates in them are in the same no_of_replicates class.

In [1]:
import os
import requests
import pickle
import pandas as pd
import numpy as np
import re
from os import walk
from collections import Counter
import random
import shutil
from statistics import median
import cmapPy.pandasGEXpress.parse_gct as pg
from cmapPy.pandasGEXpress.parse import parse

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

### - Load in Level 4 Datasets generated from `calculate_median_scores_notebook`

In [2]:
L1000_level4_path = "L1000_lvl4_cpd_replicate_datasets"

In [3]:
df_level4 = pd.read_csv(os.path.join(L1000_level4_path, 'L1000_level4W_cpd_replicates.csv.gz'), 
                        compression='gzip',low_memory = False)
df_cpd_med_scores = pd.read_csv(os.path.join(L1000_level4_path, 'cpd_replicate_median_scores_w.csv'))

In [4]:
df_cpd_med_scores = df_cpd_med_scores.set_index('cpd').rename_axis(None, axis=0).copy()

In [5]:
df_cpd_med_scores.shape

(1330, 7)

In [6]:
df_level4.shape

(27837, 988)

In [7]:
def get_cpds_replicates(df, df_lvl4):
    """
    This function returns all replicates id/names found in each compound 
    and in all doses(1-6)
    """
    
    dose_list = list(set(df_lvl4['dose'].unique().tolist()))[1:7]
    replicates_in_all = []
    cpds_replicates = {}
    for dose in dose_list:
        rep_list = []
        df_doses = df_lvl4[df_lvl4['dose'] == dose].copy()
        for cpd in df.index:
            replicate_names = df_doses[df_doses['pert_iname'] == cpd]['replicate_id'].values.tolist()
            rep_list += replicate_names
            if cpd not in cpds_replicates:
                cpds_replicates[cpd] = [replicate_names]
            else:
                cpds_replicates[cpd] += [replicate_names]
        replicates_in_all.append(rep_list)
        
    return replicates_in_all, cpds_replicates

In [8]:
replicates_in_all, cpds_replicates = get_cpds_replicates(df_cpd_med_scores, df_level4)

In [9]:
def get_replicates_classes_per_dose(df, df_lvl4, cpds_replicates):
    
    """
    This function gets all replicates ids for each distinct 
    no_of_replicates (i.e. number of replicates per cpd) class per dose (1-6)
    
    Returns replicate_class_dict dictionary, with no_of_replicate classes as the keys, 
    and all the replicate_ids for each no_of_replicate class as the values
    """
    
    df['replicate_id'] = list(cpds_replicates.values())
    dose_list = list(set(df_lvl4['dose'].unique().tolist()))[1:7]
    replicate_class_dict = {}
    for dose in dose_list:
        for size in df['no_of_replicates'].unique():
            rep_lists = []
            for idx in range(df[df['no_of_replicates'] == size].shape[0]):
                rep_ids = df[df['no_of_replicates'] == size]['replicate_id'].values.tolist()[idx][dose-1]
                rep_lists += rep_ids
            if size not in replicate_class_dict:
                replicate_class_dict[size] = [rep_lists]
            else:
                replicate_class_dict[size] += [rep_lists]
                
    return replicate_class_dict

In [10]:
cpd_replicate_class_dict = get_replicates_classes_per_dose(df_cpd_med_scores, df_level4, cpds_replicates)

In [11]:
cpd_replicate_class_dict.keys()

dict_keys([3, 2, 4, 6])

In [12]:
def check_similar_replicates(replicates, dose, cpd_dict):
    """This function checks if two replicates are of the same compounds"""
    
    for x in range(len(replicates)):
        for y in range(x+1, len(replicates)):
            for kys in cpd_dict:
                if all(i in cpd_dict[kys][dose-1] for i in [replicates[x], replicates[y]]):
                    return True
    return False

In [13]:
def get_random_replicates(all_replicates, no_of_replicates, dose, replicates_ids, cpd_replicate_dict):
    """
    This function return a list of random replicates that are not of the same compounds
    or found in the current cpd's size list
    """
    while (True):
        random_replicates = random.sample(all_replicates, no_of_replicates)
        if not (any(rep in replicates_ids for rep in random_replicates) & 
                (check_similar_replicates(random_replicates, dose, cpd_replicate_dict))):
            break
    return random_replicates

In [14]:
def get_null_distribution_replicates(cpd_replicate_class_dict, dose_list, replicates_lists, 
                                     cpd_replicate_dict, rand_num = 1000):
    
    """
    This function returns a null distribution dictionary, with no_of_replicates(replicate class) 
    as the keys and 1000 lists of randomly selected replicate combinations as the values
    for each no_of_replicates class per DOSE(1-6)
    """
    random.seed(1903)
    null_distribution_reps = {}
    for dose in dose_list:
        for replicate_class in cpd_replicate_class_dict:
            replicates_ids = cpd_replicate_class_dict[replicate_class][dose-1]
            replicate_list = []
            for idx in range(rand_num):
                start_again = True
                while (start_again):
                    rand_cpds = get_random_replicates(replicates_lists[dose-1], replicate_class, dose, 
                                                      replicates_ids, cpd_replicate_dict)
                    if rand_cpds not in replicate_list:
                        start_again = False
                replicate_list.append(rand_cpds)
            if replicate_class not in null_distribution_reps:
                null_distribution_reps[replicate_class] = [replicate_list]
            else:
                null_distribution_reps[replicate_class] += [replicate_list]
    
    return null_distribution_reps

In [15]:
dose_list = list(set(df_level4['dose'].unique().tolist()))[1:7]
null_distribution_replicates = get_null_distribution_replicates(cpd_replicate_class_dict, dose_list, 
                                                                replicates_in_all, cpds_replicates)

In [16]:
def save_to_pickle(null_distribution, path, file_name):
    """This function saves the null distribution replicates ids into a pickle file"""
    
    if not os.path.exists(path):
        os.mkdir(path)
        
    with open(os.path.join(path, file_name), 'wb') as handle:
        pickle.dump(null_distribution, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
save_to_pickle(null_distribution_replicates, L1000_level4_path, 'null_distribution_w.pickle')

In [18]:
##load the null_distribution_moa from pickle
with open(os.path.join(L1000_level4_path, 'null_distribution_w.pickle'), 'rb') as handle:
    null_distribution_replicates = pickle.load(handle)

In [19]:
def assert_null_distribution(null_distribution_reps, dose_list):
    
    """
    This function assert that each of the list in the 1000 lists of 
    random replicate combination (per dose) for each cpd size are distinct with no duplicates
    """
    
    duplicates_reps = {}
    for dose in dose_list:
        for keys in null_distribution_reps:
            null_dist = null_distribution_reps[keys][dose-1]
            for reps in null_dist:
                dup_reps = []
                new_list = list(filter(lambda x: x != reps, null_dist))
                if (len(new_list) != len(null_dist) - 1):
                    dup_reps.append(reps)
            if dup_reps:
                if keys not in duplicates_reps:
                    duplicates_reps[keys] = [dup_reps]
                else:
                    duplicates_reps[keys] += [dup_reps]
    return duplicates_reps

In [20]:
duplicate_replicates = assert_null_distribution(null_distribution_replicates, dose_list)

In [21]:
duplicate_replicates ##no duplicates

{}

In [22]:
def calc_null_dist_median_scores(df, dose_num, replicate_lists):
    """
    This function calculate the median of the correlation 
    values for each list in the 1000 lists of random replicate 
    combination for each no_of_replicate class per dose
    """
    df_dose = df[df['dose'] == dose_num].copy()
    df_dose = df_dose.set_index('replicate_id').rename_axis(None, axis=0)
    df_dose.drop(['Metadata_broad_sample', 'pert_id', 'dose', 'pert_idose', 
                  'pert_iname', 'moa', 'sig_id', 'det_plate', 'det_well'], axis = 1, inplace = True)
    median_corr_list = []
    for rep_list in replicate_lists:
        df_reps = df_dose.loc[rep_list].copy()
        reps_corr = df_reps.astype('float64').T.corr(method = 'spearman').values
        median_corr_val = median(list(reps_corr[np.triu_indices(len(reps_corr), k = 1)]))
        median_corr_list.append(median_corr_val)
    return median_corr_list

In [23]:
def get_null_dist_median_scores(null_distribution_cpds, dose_list, df):
    """ 
    This function calculate the median correlation scores for all 
    1000 lists of randomly combined compounds for each no_of_replicate class 
    across all doses (1-6)
    """
    null_distribution_medians = {}
    for key in null_distribution_cpds:
        median_score_list = []
        for dose in dose_list:
            replicate_median_scores = calc_null_dist_median_scores(df, dose, null_distribution_cpds[key][dose-1])
            median_score_list.append(replicate_median_scores)
        null_distribution_medians[key] = median_score_list
    return null_distribution_medians

In [24]:
null_distribution_medians = get_null_dist_median_scores(null_distribution_replicates, dose_list, df_level4)

In [25]:
def compute_dose_median_scores(null_dist_medians, dose_list):
    """
    Align median scores per dose, this function return a dictionary, 
    with keys as dose numbers and values as all median scores for each dose
    """
    median_scores_per_dose = {}
    for dose in dose_list:
        median_list = []
        for keys in null_distribution_medians:
            dose_median_list = null_distribution_medians[keys][dose-1]
            median_list += dose_median_list
        median_scores_per_dose[dose] = median_list
    return median_scores_per_dose

In [26]:
dose_null_medians = compute_dose_median_scores(null_distribution_medians, dose_list)

In [27]:
#save the null_distribution_medians_per_dose to pickle
save_to_pickle(dose_null_medians, L1000_level4_path, 'null_dist_medians_per_dose_w.pickle')

**A P value can be computed nonparametrically by evaluating the probability of random replicates of different compounds having median similarity value greater than replicates of the same compounds.**

In [28]:
def get_p_value(median_scores_list, df, dose_name, cpd_name):
    """
    This function calculate the p-value from the 
    null_distribution median scores for each compound
    """
    actual_med = df.loc[cpd_name, dose_name]
    p_value = np.sum(median_scores_list >= actual_med) / len(median_scores_list)
    return p_value

In [29]:
def get_moa_p_vals(null_dist_median, dose_list, df_med_values):
    """
    This function returns a dict, with compounds as the keys and the compound's 
    p-values for each dose (1-6) as the values
    """
    null_p_vals = {}
    for key in null_dist_median:
        df_cpd_size = df_med_values[df_med_values['no_of_replicates'] == key]
        for cpd in df_cpd_size.index:
            dose_p_values = []
            for num in dose_list:
                dose_name = 'dose_' + str(num)
                cpd_p_value = get_p_value(null_dist_median[key][num-1], df_cpd_size, dose_name, cpd)
                dose_p_values.append(cpd_p_value)
            null_p_vals[cpd] = dose_p_values
    sorted_null_p_vals = {key:value for key, value in sorted(null_p_vals.items(), key=lambda item: item[0])}
    return sorted_null_p_vals

In [30]:
null_p_vals = get_moa_p_vals(null_distribution_medians, dose_list, df_cpd_med_scores)

In [31]:
df_null_p_vals = pd.DataFrame.from_dict(null_p_vals, orient='index', columns = ['dose_' + str(x) for x in dose_list])

In [32]:
df_null_p_vals['no_of_replicates'] = df_cpd_med_scores['no_of_replicates']

In [33]:
df_null_p_vals.head(10)

,dose_1,dose_2,dose_3,dose_4,dose_5,dose_6,no_of_replicates
17-hydroxyprogesterone-caproate,0.921,0.113,0.042,0.000,0.013,0.121,3
2-iminobiotin,0.899,0.014,0.549,0.762,0.100,0.618,2
3-amino-benzamide,0.009,0.327,0.400,0.502,0.196,0.009,3
3-deazaadenosine,0.224,0.521,0.325,0.224,0.505,0.300,2
ABT-737,0.165,0.904,0.101,0.403,0.048,0.002,3
AICA-ribonucleotide,0.187,0.059,0.345,0.015,0.362,0.032,3
AKT-inhibitor-1-2,0.586,0.050,0.096,0.088,0.530,0.537,3
ALX-5407,0.036,0.111,0.003,0.088,0.129,0.010,3
AS-605240,0.972,0.033,0.841,0.810,0.033,0.077,2
AT-7519,0.002,0.197,0.043,0.000,0.000,0.000,3


In [34]:
def save_to_csv(df, path, file_name):
    """saves dataframes to csv"""
    
    if not os.path.exists(path):
        os.mkdir(path)
    
    df.to_csv(os.path.join(path, file_name), index = False)

In [35]:
save_to_csv(df_null_p_vals.reset_index().rename({'index':'cpd'}, axis = 1), L1000_level4_path, 
            'cpd_replicate_p_values_w.csv')